Setup Instructions
==========

These instructions need to be run once to load the requisite libraries for the tutorial.

Jupyter has the concept of _kernels_, which are independent execution environments. They don't
even have to be Python, kernels for other languages exist as well.

By loading a separate kernel for each project, we avoid the complication of different
components/projects having weird interactions, ultimately helping reproducibility.

We first produce a new virtualenv with the libraries we require, then we teach Jupyter
about this new environment with the ipython executable

In [1]:
%%bash
#!/bin/bash

CMSSW_VER="CMSSW_10_1_0"
KERNEL_NAME="machine-learning-hats"

set -e
# Get the CMSSW libraries (specifically ROOT)
SCRAM_ARCH=$(ls -d /cvmfs/cms.cern.ch/$(/cvmfs/cms.cern.ch/common/cmsos)*/cms/cmssw/${CMSSW_VER} | tail -n 1 | awk -F / '{ print $4 }')
export SCRAM_ARCH
source /cvmfs/cms.cern.ch/cmsset_default.sh
if [ ! -d cmssw-env ]; then
    scramv1 project --name cmssw-env CMSSW $CMSSW_VER 
fi
cd cmssw-env
eval `scramv1 runtime -sh`

# Make a wrapper script to load CMSSW python
cat << 'EOF' > bin/python_wrapper.sh
#!/bin/bash
DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )"
source /cvmfs/cms.cern.ch/cmsset_default.sh
cd $DIR
eval `scramv1 runtime -sh`
cd -
exec python "$@"
EOF
chmod +x bin/python_wrapper.sh

# Create the kernel
mkdir -p "$HOME/.local/share/jupyter/kernels/$KERNEL_NAME"
cat << EOF > "$HOME/.local/share/jupyter/kernels/$KERNEL_NAME/kernel.json"
{
 "display_name": "$KERNEL_NAME", 
 "language": "python", 
 "argv": [
  "$PWD/bin/python_wrapper.sh", 
  "-m", 
  "ipykernel_launcher", 
  "-f", 
  "{connection_file}"
 ]
}
EOF

# Report OK
echo "Loaded $CMSSW_VERSION into $KERNEL_NAME!"

Loaded CMSSW_10_1_0 into machine-learning-hats!


bash: line 1: fg: no job control
perl: warning: Setting locale failed.
perl: warning: Please check that your locale settings:
	LANGUAGE = (unset),
	LC_ALL = (unset),
	LANG = "C.UTF-8"
    are supported and installed on your system.
perl: warning: Falling back to the standard locale ("C").
perl: warning: Setting locale failed.
perl: warning: Please check that your locale settings:
	LANGUAGE = (unset),
	LC_ALL = (unset),
	LANG = "C.UTF-8"
    are supported and installed on your system.
perl: warning: Falling back to the standard locale ("C").


Results
=======

If successful, you should see something similar to the following:

```
New python executable in /home/meloam/hats-template/hats-template/bin/python2
Also creating executable in /home/meloam/hats-template/hats-template/bin/python
Installing setuptools, pip, wheel...done.
/home/meloam/hats-template/hats-template/bin/pip
Collecting numpy==1.14.3
  Using cached https://files.pythonhosted.org/packages/c0/e7/08f059a00367fd613e4f2875a16c70b6237268a1d6d166c6d36acada8301/numpy-1.14.3-cp27-cp27mu-manylinux1_x86_64.whl
<snip>
Installed kernelspec hats-template in /home/meloam/.local/share/jupyter/kernels/hats-template
```

The new kernel you just made will then show up in the various Jupyter dropdowns, allowing you to use it for different notebooks